In [25]:
import pandas as pd
import numpy as np
from sklearn import tree
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
import features
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [12]:
df = pd.read_csv('all_data_combined.csv')
df.head()

,subject_id,age,urea_n_min,urea_n_max,urea_n_mean,resprate_min,resprate_max,resprate_mean,glucose_min,glucose_max,...,oasis,lods,gender_F,gender_M,marital_status_DIVORCED,marital_status_MARRIED,marital_status_SEPARATED,marital_status_SINGLE,marital_status_WIDOWED,death
0,15057,58.831224,16.0,37.0,26.111111,10.0,52,20.104478,67.0,405.0,...,36.5,5.5,0,1,0,1,0,0,0,1
1,79262,81.618606,10.0,15.0,13.500000,11.0,31,18.704225,119.0,348.0,...,48.0,4.0,0,1,0,1,0,0,0,1
2,77191,79.102744,16.0,56.0,35.500000,20.0,31,25.744681,90.0,188.0,...,35.0,6.0,1,0,1,0,0,0,0,0
3,84966,88.232043,8.0,27.0,17.692308,10.0,51,17.770833,92.0,271.0,...,40.5,4.0,1,0,0,0,0,1,0,1
4,94997,90.000000,38.0,48.0,42.250000,4.0,40,26.360000,62.0,135.0,...,35.0,5.0,1,0,0,0,0,1,0,0


In [13]:
df.shape

(2152, 168)

In [14]:

model = RandomForestClassifier()
model.fit(df.iloc[0:1800,1:df.shape[1]-1], df.iloc[0:1800,df.shape[1]-1]) 
model.score(df.iloc[1800:,1:df.shape[1]-1], df.iloc[1800:,df.shape[1]-1])


0.8153409090909091

In [15]:
features=df.columns[1:len(df.columns)-1]
def getFeatures(weights):
    df_w= pd.DataFrame(features)
    df_w.columns =['features']
    df_w['weight']=weights
    return df_w

In [16]:
df_weight = getFeatures(model.feature_importances_)
df_weight.sort_values(by='weight',inplace=True,ascending=False)
df_weight

df_weight.to_csv("randomForest.csv",index = False)

In [34]:
def randomForest(df):
    
    kf = KFold(n_splits=3, random_state=0)
    
    result = []
    
    for train, test in kf.split(df):

        train_data = df.iloc[train,:]
        test_data =  df.iloc[test,:]

        trainx = train_data.iloc[:,1:(train_data.shape[1]-2)]
        trainy = train_data.iloc[:,train_data.shape[1]-1]

        testx = test_data.iloc[:,1:(test_data.shape[1]-2)]
        testy = test_data.iloc[:,test_data.shape[1]-1]

        model = RandomForestClassifier(n_estimators=16, max_depth= 4, min_samples_leaf=32)
        model.fit(trainx, trainy)
        result.append(model.score(testx, testy))
               
    return np.average(result)

In [35]:
result=randomForest(df)
print (result)

0.7723051660884552


In [26]:
def randomForestPara(df,estimators):
    
    kf = KFold(n_splits=3, random_state=0)
    
    result = []
    
    for train, test in kf.split(df):

        train_data = df.iloc[train,:]
        test_data =  df.iloc[test,:]

        trainx = train_data.iloc[:,1:(train_data.shape[1]-2)]
        trainy = train_data.iloc[:,train_data.shape[1]-1]

        testx = test_data.iloc[:,1:(test_data.shape[1]-2)]
        testy = test_data.iloc[:,test_data.shape[1]-1]

        model = RandomForestClassifier(n_estimators=estimators, max_depth= 4, min_samples_leaf=32)
        model.fit(trainx, trainy)
        result.append(model.score(testx, testy))
               
    return np.average(result)

In [27]:
estimators = [1,2,4,8,16,32,64]
scores = []
for row in range (len(estimators)):
    scores.append(randomForestPara(df,estimators = estimators[row]))

In [38]:
plt.plot(estimators,scores,'ro-')
plt.xlabel('No of estimators')
plt.ylabel('Accuracy')
plt.savefig("random_for_estimators.png")
plt.close()

In [37]:
scores

[0.6965471782898153,
 0.735591659770865,
 0.7462817708677313,
 0.7537227615839753,
 0.7727687711487435,
 0.7727726561073491,
 0.7792767243065025]